In [257]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import os
from sklearn.metrics import r2_score

In [258]:
#obtain list of dates
directory = "Counties"
list_dates = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        list_dates.append(filename)

In [274]:
# get list of industries

industries = pd.read_csv(f'Counties/{list_dates[0]}').columns.tolist()[1:]

In [302]:
#counties to drop
drop_counties = ['Inyo', 'Colusa', 'Imperial', 'Alpine']

results = {}

#iterate through industries

for industry in industries:
    
    final_df = pd.DataFrame()

    for count in list_dates:
        county = count.split('.')[0]

        if county == 'Contra Costa':
            county = 'Contra-Costa'

        if county in drop_counties:
            continue
            
        # get housing data

        housing = pd.read_csv('UnsoldInventory.csv')[[county, 'Year']]

        housing['Shifted County'] = housing[county].shift(1)
        
        # obtain change from previous year of each county data

        housing['Change'] = (housing[county] - housing['Shifted County']) / housing[county]
        
        housing = housing.replace(0, np.nan)

        housing.dropna(inplace=True)

        transfer_county = housing[['Change', 'Year']]
        
        # read in county gdp data

        data = pd.read_csv(f'Counties/{list_dates[0]}').sort_values(by='Unnamed: 0')
        
        # shift industry data

        data['Shifted Industry'] = data[industry].shift(1)
        
        data = data.replace(0, np.nan)

        data.dropna(inplace=True)
        
        # obtain change of industry data year over year

        data[f'Change {industry}'] = (data[industry] - data['Shifted Industry']) / data[industry]

        data['Year'] = data['Unnamed: 0']

        transfer = data[['Year', f'Change {industry}']]

        data_df = pd.merge(transfer_county, transfer, on='Year')

        final_df = pd.concat([data_df, final_df])
        
    # create linear regression between industry gdp data and change in housing data
        
    X = final_df[f'Change {industry}'].values.reshape(-1,1)

    y = final_df['Change']
    
    model = LinearRegression()

    model.fit(X, y)

    y_pred = model.predict(X)
    
    results[industry] = {'Coef': model.coef_[0], 'R^2': r2_score(y, y_pred)}
    
    

In [303]:
# displays coef and R^2

pd.DataFrame.from_dict(results, orient='index')

,Coef,R^2
Private industries,1.020221,0.009074
"Agriculture, forestry, fishing, and hunting",-0.252770,0.017376
Mining,-0.121172,0.004892
Utilities,0.022543,0.000139
Construction,0.216180,0.005445
Manufacturing,-1.292276,0.008059
Durable goods manufacturing,1.284922,0.011420
Nondurable goods manufacturing,-1.206138,0.059695
Wholesale trade,1.384983,0.057218
Retail trade,-0.393668,0.002789


In [277]:
drop_counties = ['Inyo', 'Colusa', 'Imperial', 'Alpine', 'Los Angeles']

In [280]:
results = {}

for industry in industries:
    
    final_df = pd.DataFrame()

    for count in list_dates:
        county = count.split('.')[0]

        if county == 'Contra Costa':
            county = 'Contra-Costa'

        if county in drop_counties:
            continue

        housing = pd.read_csv('MedianTime.csv')[[county, 'Year']]

        housing['Shifted County'] = housing[county].shift(1)

        housing['Change'] = (housing[county] - housing['Shifted County']) / housing[county]
        
        housing = housing.replace(0, np.nan)

        housing.dropna(inplace=True)

        transfer_county = housing[['Change', 'Year']]

        data = pd.read_csv(f'Counties/{list_dates[0]}').sort_values(by='Unnamed: 0')

        data['Shifted Industry'] = data[industry].shift(1)
        
        data = data.replace(0, np.nan)

        data.dropna(inplace=True)

        data[f'Change {industry}'] = (data[industry] - data['Shifted Industry']) / data[industry]

        data['Year'] = data['Unnamed: 0']

        transfer = data[['Year', f'Change {industry}']]

        data_df = pd.merge(transfer_county, transfer, on='Year')

        final_df = pd.concat([data_df, final_df])
        
    X = final_df[f'Change {industry}'].values.reshape(-1,1)

    y = final_df['Change']
    
    model = LinearRegression()

    model.fit(X, y)

    y_pred = model.predict(X)
    
    results[industry] = {'Coef': model.coef_[0], 'R^2': r2_score(y, y_pred)}
    
    

In [281]:
pd.DataFrame.from_dict(results, orient='index')

,Coef,R^2
Private industries,-1.028952,0.011442
"Agriculture, forestry, fishing, and hunting",-0.025418,0.000209
Mining,-0.181287,0.013177
Utilities,-0.020652,0.000144
Construction,-0.205155,0.006616
Manufacturing,-2.498455,0.036903
Durable goods manufacturing,-0.788901,0.005253
Nondurable goods manufacturing,-0.745446,0.028587
Wholesale trade,0.417827,0.006457
Retail trade,-1.116394,0.029750


In [265]:
import numpy as np

In [300]:
results = {}

for industry in industries:
    
    final_df = pd.DataFrame()

    for count in list_dates:
        county = count.split('.')[0]

        if county == 'Contra Costa':
            county = 'Contra-Costa'

        if county in drop_counties:
            continue

        housing = pd.read_csv('PercentChangeSales.csv')[[county, 'Year']]

        housing['Shifted County'] = housing[county].shift(1)

        housing['Change'] = housing[county]
        
        housing = housing.replace(0, np.nan)

        housing.dropna(inplace=True)

        transfer_county = housing[['Change', 'Year']]

        data = pd.read_csv(f'Counties/{list_dates[0]}').sort_values(by='Unnamed: 0')

        data['Shifted Industry'] = data[industry].shift(1)
        
        data = data.replace(0, np.nan)

        data.dropna(inplace=True)

        data[f'Change {industry}'] = (data[industry] - data['Shifted Industry']) / data[industry]

        data['Year'] = data['Unnamed: 0']

        transfer = data[['Year', f'Change {industry}']]

        data_df = pd.merge(transfer_county, transfer, on='Year')

        final_df = pd.concat([data_df, final_df])
        
    X = final_df[f'Change {industry}'].values.reshape(-1,1)

    y = final_df['Change']
    
    model = LinearRegression()

    model.fit(X, y)

    y_pred = model.predict(X)
    
    results[industry] = {'Coef': model.coef_[0], 'R^2': r2_score(y, y_pred)}
    
    

In [301]:
pd.DataFrame.from_dict(results, orient='index')

,Coef,R^2
Private industries,-3.008891,0.063647
"Agriculture, forestry, fishing, and hunting",0.506605,0.054287
Mining,-0.166328,0.007080
Utilities,0.193026,0.008547
Construction,-0.490534,0.026227
Manufacturing,-1.598103,0.010137
Durable goods manufacturing,-1.195374,0.007866
Nondurable goods manufacturing,-0.062582,0.000140
Wholesale trade,-0.889920,0.019067
Retail trade,-1.061181,0.018641


In [77]:
final_df

,Change,Year,"Change Other services, except government"
0,0.128282,2002,0.084769
1,0.078162,2003,0.006945
2,0.186794,2004,0.045409
3,0.089058,2005,0.082327
4,-0.006737,2006,0.036243
...,...,...,...
15,0.061157,2017,0.026489
16,0.021827,2018,0.065205
17,0.055725,2019,0.032524
18,0.102740,2020,-0.168577


In [284]:
drop_counties = ['Inyo', 'Colusa', 'Imperial', 'Alpine']

results = {}

for industry in industries:
    
    final_df = pd.DataFrame()

    for count in list_dates:
        county = count.split('.')[0]

        if county == 'Contra Costa':
            county = 'Contra-Costa'

        if county in drop_counties:
            continue

        housing = pd.read_csv('MedianPricesExisting.csv')[[county, 'Year']]

        housing['Shifted County'] = housing[county].shift(1)

        housing['Change'] = (housing[county] - housing['Shifted County']) / housing[county]
        
        housing = housing.replace(0, np.nan)

        housing.dropna(inplace=True)

        transfer_county = housing[['Change', 'Year']]

        data = pd.read_csv(f'Counties/{list_dates[0]}').sort_values(by='Unnamed: 0')

        data['Shifted Industry'] = data[industry].shift(1)
        
        data = data.replace(0, np.nan)

        data.dropna(inplace=True)

        data[f'Change {industry}'] = (data[industry] - data['Shifted Industry']) / data[industry]

        data['Year'] = data['Unnamed: 0']

        transfer = data[['Year', f'Change {industry}']]

        data_df = pd.merge(transfer_county, transfer, on='Year')

        final_df = pd.concat([data_df, final_df])
        
    X = final_df[f'Change {industry}'].values.reshape(-1,1)

    y = final_df['Change']
    
    model = LinearRegression()

    model.fit(X, y)

    y_pred = model.predict(X)
    
    results[industry] = {'Coef': model.coef_[0], 'R^2': r2_score(y, y_pred)}
    
    
pd.DataFrame.from_dict(results, orient='index')

,Coef,R^2
Private industries,3.018251,0.429622
"Agriculture, forestry, fishing, and hunting",-0.207806,0.061657
Mining,0.159200,0.043529
Utilities,-0.092098,0.013031
Construction,0.946564,0.654223
Manufacturing,0.849117,0.018999
Durable goods manufacturing,0.991304,0.036166
Nondurable goods manufacturing,-0.131979,0.004116
Wholesale trade,0.766727,0.095013
Retail trade,2.224242,0.548254


# Multiple Linear Regression

In [285]:
drop_counties = ['Inyo', 'Colusa', 'Imperial', 'Alpine']

results = {}

final_df = pd.DataFrame()

for count in list_dates:
    county = count.split('.')[0]

    if county == 'Contra Costa':
        county = 'Contra-Costa'

    if county in drop_counties:
        continue
        
        
    # obtain change in housing data

    housing = pd.read_csv('MedianPricesExisting.csv')[[county, 'Year']]

    housing['Shifted County'] = housing[county].shift(1)

    housing['Change'] = (housing[county] - housing['Shifted County']) / housing[county]

    housing = housing.replace(0, np.nan)

    housing.dropna(inplace=True)

    transfer_county = housing[['Change', 'Year']]
    
    # obtain change year over year with selected industries

    data = pd.read_csv(f'Counties/{list_dates[0]}').sort_values(by='Unnamed: 0')

    data['Construction Industry'] = data['Construction'].shift(1)
    
    data['Finance and insurance Industry'] = data['Finance and insurance'].shift(1)
    
    data['Retail trade Industry'] = data['Retail trade'].shift(1)
    
    data['Information Industry'] = data['Information'].shift(1)
    
    data['Finance, insurance, real estate, rental, and leasing Industry'] = data['Finance, insurance, real estate, rental, and leasing'].shift(1)

    data = data.replace(0, np.nan)

    data.dropna(inplace=True)

    data[f'Change Construction'] = (data['Construction'] - data['Construction Industry']) / data['Construction']
    
    data[f'Change Finance and insurance'] = (data['Finance and insurance'] - data['Finance and insurance Industry']) / data['Finance and insurance']
    
    data[f'Change Retail trade'] = (data['Retail trade'] - data['Retail trade Industry']) / data['Retail trade']
    
    data[f'Change Information'] = (data['Information'] - data['Information Industry']) / data['Information']
    
    data[f'Change Finance, insurance, real estate, rental, and leasing'] = (data['Finance, insurance, real estate, rental, and leasing'] - data['Finance, insurance, real estate, rental, and leasing Industry']) / data['Finance, insurance, real estate, rental, and leasing']

    data['Year'] = data['Unnamed: 0']

    transfer = data[['Year', 'Change Finance and insurance', 'Change Construction', 'Change Retail trade', 'Change Information', 'Change Finance, insurance, real estate, rental, and leasing']]

    data_df = pd.merge(transfer_county, transfer, on='Year')

    final_df = pd.concat([data_df, final_df])

In [296]:
# Perform multiple linear regression

X = final_df[['Change Construction', 'Change Information']].values.reshape(-2,2)

y = final_df['Change']

model = LinearRegression()

model.fit(X, y)

y_pred = model.predict(X)

results[industry] = {'Coef': model.coef_[0], 'R^2': r2_score(y, y_pred)}

In [297]:
# displays coef and R^2

pd.DataFrame.from_dict(results, orient='index')

,Coef,R^2
"Other services, except government",0.705842,0.682052


In [298]:
#displays adjusted RMSE
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y, y_pred))

rmse 

0.08677140412476646

In [299]:
# displays adjusted R^2

n = len(y)
p = X.shape[1]
adjusted_r_squared = 1 - (1 - r2_score(y, y_pred)) * (n - 1) / (n - p - 1)

adjusted_r_squared

0.6807651040370052

In [146]:
drop_counties = ['Inyo', 'Colusa', 'Imperial', 'Alpine', 'Los Angeles']

In [166]:
results = {}

final_df = pd.DataFrame()

for count in list_dates:
    county = count.split('.')[0]

    if county == 'Contra Costa':
        county = 'Contra-Costa'

    if county in drop_counties:
        continue

    housing = pd.read_csv('MedianTime.csv')[[county, 'Year']]

    housing['Shifted County'] = housing[county].shift(1)

    housing['Change'] = (housing[county] - housing['Shifted County']) / housing[county]

    housing = housing.replace(0, np.nan)

    housing.dropna(inplace=True)

    transfer_county = housing[['Change', 'Year']]

    data = pd.read_csv(f'Counties/{list_dates[0]}').sort_values(by='Unnamed: 0')

    data['Arts, entertainment, and recreation Industry'] = data['Arts, entertainment, and recreation'].shift(1)

    data['Accommodation and food services Industry'] = data['Accommodation and food services'].shift(1)

    data['Other services, except government Industry'] = data['Other services, except government'].shift(1)

    data['Transportation and warehousing Industry'] = data['Transportation and warehousing'].shift(1)

    data['Real estate and rental and leasing Industry'] = data['Real estate and rental and leasing'].shift(1)

    data = data.replace(0, np.nan)

    data.dropna(inplace=True)

    data[f'Change Arts, entertainment, and recreation'] = (data['Arts, entertainment, and recreation'] - data['Arts, entertainment, and recreation Industry']) / data['Arts, entertainment, and recreation']

    data[f'Change Accommodation and food services'] = (data['Accommodation and food services'] - data['Accommodation and food services Industry']) / data['Accommodation and food services']

    data[f'Change Other services, except government'] = (data['Other services, except government'] - data['Other services, except government Industry']) / data['Other services, except government']

    data[f'Change Transportation and warehousing'] = (data['Transportation and warehousing'] - data['Transportation and warehousing Industry']) / data['Transportation and warehousing']

    data[f'Change Real estate and rental and leasing'] = (data['Real estate and rental and leasing Industry']) / data['Real estate and rental and leasing']

    data['Year'] = data['Unnamed: 0']

    transfer = data[['Year', 'Change Arts, entertainment, and recreation', 'Change Accommodation and food services', 'Change Other services, except government', 'Change Real estate and rental and leasing', 'Change Transportation and warehousing']]


    data_df = pd.merge(transfer_county, transfer, on='Year')

    final_df = pd.concat([data_df, final_df])




In [171]:
X = final_df[['Change Arts, entertainment, and recreation', 'Change Accommodation and food services', 'Change Other services, except government', 'Change Transportation and warehousing']].values.reshape(-4,4)

y = final_df['Change']

model = LinearRegression()

model.fit(X, y)

y_pred = model.predict(X)

results[industry] = {'Coef': model.coef_[0], 'R^2': r2_score(y, y_pred)}

In [172]:
pd.DataFrame.from_dict(results, orient='index')

,Coef,R^2
"Other services, except government",0.665403,0.383875


In [173]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y, y_pred))

rmse 

0.4739842706590917

In [174]:
n = len(y)
p = X.shape[1]
adjusted_r_squared = 1 - (1 - r2_score(y, y_pred)) * (n - 1) / (n - p - 1)

adjusted_r_squared

0.3779931873571858

In [184]:
results = {}

final_df = pd.DataFrame()

drop_counties = ['Inyo', 'Colusa', 'Imperial', 'Alpine']

for count in list_dates:
    county = count.split('.')[0]

    if county == 'Contra Costa':
        county = 'Contra-Costa'

    if county in drop_counties:
        continue

    housing = pd.read_csv('UnsoldInventory.csv')[[county, 'Year']]

    housing['Shifted County'] = housing[county].shift(1)

    housing['Change'] = (housing[county] - housing['Shifted County']) / housing[county]

    housing = housing.replace(0, np.nan)

    housing.dropna(inplace=True)

    transfer_county = housing[['Change', 'Year']]

    data = pd.read_csv(f'Counties/{list_dates[0]}').sort_values(by='Unnamed: 0')

    data['Transportation and warehousing Industry'] = data['Transportation and warehousing'].shift(1)
    
    data['Accommodation and food services Industry'] = data['Accommodation and food services'].shift(1)

    data['Other services, except government Industry'] = data['Other services, except government'].shift(1)
    
    data['Information Industry'] = data['Information'].shift(1)
    
    data['Private industries Industry'] = data['Private industries'].shift(1)

    data = data.replace(0, np.nan)

    data.dropna(inplace=True)

    data[f'Change Transportation and warehousing'] = (data['Transportation and warehousing'] - data['Transportation and warehousing Industry']) / data['Transportation and warehousing']
    
    data[f'Change Accommodation and food services'] = (data['Accommodation and food services'] - data['Accommodation and food services Industry']) / data['Accommodation and food services']

    data[f'Change Other services, except government'] = (data['Other services, except government'] - data['Other services, except government Industry']) / data['Other services, except government']
    
    data[f'Change Information'] = (data['Information'] - data['Information Industry']) / data['Information']
    
    data[f'Change Private industries'] = (data['Private industries'] - data['Private industries Industry']) / data['Private industries']

    data['Year'] = data['Unnamed: 0']

    transfer = data[['Year', 'Change Other services, except government', 'Change Transportation and warehousing', 'Change Accommodation and food services', 'Change Information', 'Change Private industries']]

    data_df = pd.merge(transfer_county, transfer, on='Year')

    final_df = pd.concat([data_df, final_df])


In [189]:
X = final_df[['Change Accommodation and food services', 'Change Other services, except government', 'Change Transportation and warehousing', 'Change Information']].values.reshape(-4,4)

y = final_df['Change']

model = LinearRegression()

model.fit(X, y)

y_pred = model.predict(X)

results[industry] = {'Coef': model.coef_[0], 'R^2': r2_score(y, y_pred)}

In [190]:
pd.DataFrame.from_dict(results, orient='index')

,Coef,R^2
"Other services, except government",-0.888676,0.265876


In [187]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y, y_pred))

rmse 

0.46322894862359326

In [188]:
n = len(y)
p = X.shape[1]
adjusted_r_squared = 1 - (1 - r2_score(y, y_pred)) * (n - 1) / (n - p - 1)

adjusted_r_squared

0.3024856121695316

In [230]:
results = {}

drop_counties = ['Inyo', 'Colusa', 'Imperial', 'Alpine', 'Los Angeles']

for industry in industries:
    
    final_df = pd.DataFrame()

    for count in list_dates:
        county = count.split('.')[0]

        if county == 'Contra Costa':
            county = 'Contra-Costa'

        if county in drop_counties:
            continue

        housing = pd.read_csv('Ratio.csv')[[county, 'Year']]

        housing['Shifted County'] = housing[county].shift(1)

        housing['Change'] = housing[county]
        
        housing = housing.replace(0, np.nan)

        housing.dropna(inplace=True)

        transfer_county = housing[['Change', 'Year']]

        data = pd.read_csv(f'Counties/{list_dates[0]}').sort_values(by='Unnamed: 0')

        data['Shifted Industry'] = data[industry].shift(1)
        
        data = data.replace(0, np.nan)

        data.dropna(inplace=True)

        data[f'Change {industry}'] = data[industry]

        data['Year'] = data['Unnamed: 0']

        transfer = data[['Year', f'Change {industry}']]

        data_df = pd.merge(transfer_county, transfer, on='Year')

        final_df = pd.concat([data_df, final_df])
        
    X = final_df[f'Change {industry}'].values.reshape(-1,1)

    y = final_df['Change']
    
    model = LinearRegression()

    model.fit(X, y)

    y_pred = model.predict(X)
    
    results[industry] = {'Coef': model.coef_[0], 'R^2': r2_score(y, y_pred)}
    
    

In [232]:
pd.DataFrame.from_dict(results, orient='index').sort_values(by='R^2', ascending=False)

,Coef,R^2
Finance and insurance,0.007041,0.267397
Administrative and waste management services,0.010548,0.261997
Retail trade,0.008744,0.245106
Professional and business services,0.001710,0.241130
Manufacturing,0.002470,0.230823
Durable goods manufacturing,0.003288,0.230459
"Professional, scientific, and technical services",0.002264,0.228587
Private industries,0.000408,0.227544
Management of companies and enterprises,0.014890,0.226155
Health care and social assistance,0.004022,0.220828


In [240]:
results = {}

final_df = pd.DataFrame()

drop_counties = ['Inyo', 'Colusa', 'Imperial', 'Alpine', 'Los Angeles']


for count in list_dates:
    county = count.split('.')[0]

    if county == 'Contra Costa':
        county = 'Contra-Costa'

    if county in drop_counties:
        continue

    housing = pd.read_csv('Ratio.csv')[[county, 'Year']]

    housing['Shifted County'] = housing[county].shift(1)

    housing['Change'] = housing[county]

    housing = housing.replace(0, np.nan)

    housing.dropna(inplace=True)

    transfer_county = housing[['Change', 'Year']]

    data = pd.read_csv(f'Counties/{list_dates[0]}').sort_values(by='Unnamed: 0')

    data['Finance and insurance Industry'] = data['Finance and insurance'].shift(1)
    
    data['Administrative and waste management services Industry'] = data['Administrative and waste management services'].shift(1)

    data['Retail trade Industry'] = data['Retail trade'].shift(1)
    
    data['Professional and business services Industry'] = data['Professional and business services'].shift(1)
    
    data['Manufacturing Industry'] = data['Manufacturing'].shift(1)

    data = data.replace(0, np.nan)

    data.dropna(inplace=True)

    data[f'Change Finance and insurance'] = (data['Finance and insurance'] - data['Finance and insurance Industry']) / data['Finance and insurance']
    
    data[f'Change Administrative and waste management services'] = (data['Administrative and waste management services'] - data['Administrative and waste management services Industry']) / data['Administrative and waste management services']

    data[f'Change Retail trade'] = (data['Retail trade'] - data['Retail trade Industry']) / data['Retail trade']
    
    data[f'Change Professional and business services'] = (data['Professional and business services'] - data['Professional and business services Industry']) / data['Professional and business services']
    
    data[f'Change Manufacturing'] = (data['Manufacturing'] - data['Manufacturing Industry']) / data['Manufacturing']

    data['Year'] = data['Unnamed: 0']

    transfer = data[['Year', 'Finance and insurance', 'Administrative and waste management services', 'Retail trade', 'Professional and business services', 'Manufacturing']]

    data_df = pd.merge(transfer_county, transfer, on='Year')

    final_df = pd.concat([data_df, final_df])


In [247]:
X = final_df[['Finance and insurance', 'Administrative and waste management services', 'Retail trade', 'Professional and business services', 'Manufacturing']].values.reshape(-5,5)

y = final_df['Change']

model = LinearRegression()

model.fit(X, y)

y_pred = model.predict(X)

results[industry] = {'Coef': model.coef_[0], 'R^2': r2_score(y, y_pred)}

In [248]:
pd.DataFrame.from_dict(results, orient='index')

,Coef,R^2
"Other services, except government",0.010648,0.289676
